In [1]:
from astropy.io import ascii
import numpy as np
import pandas as pd
import subprocess
import os

In [2]:
#Get position for a SINGLE gid in format 'x y' for using in the .feedme file position
def get_position(field, gid):
    '''
    Gets positions of objects in fitting region in pixel coord
    
    Parameters:
    -----------
    field : string
        GOODS_South or GOODS_North
    gid : float
        ID number of object
    '''
    
#     Define catalog based on the field its in
    if field == 'S':
        catalog = '/Users/rosaliaobrien/research/website/catalogs/goodss_3dhst.v4.4.cat'
        
    if field == 'N':
        catalog = '/Users/rosaliaobrien/research/website/catalogs/goodsn_3dhst.v4.4.cat'
    
    #Return x and y coordinates (with a space) of single gid
    cat = ascii.read(catalog)
    x = cat[cat['id']==gid]['x'].data[0]
    y = cat[cat['id']==gid]['y'].data[0]
#     print('G'+field+'D'+str(gid)+' coordinates:', x, y)
    coord = str(x)+' '+str(y)
    return coord

In [3]:
#Get positions for a LIST of gids so that we can determine the boundaries for the cutoutsize of the .feedme file
def get_positions(gids, fields):
    
    x=[]
    y=[]
    
    for gid, field in zip(gids, fields):
        
        if field == 'S':
            #Find catalog
            catalog = '/Users/rosaliaobrien/research/website/catalogs/goodss_3dhst.v4.4.cat'

        if field == 'N':
            #Find catalog
            catalog = '/Users/rosaliaobrien/research/website/catalogs/goodsn_3dhst.v4.4.cat'
         
        cat = ascii.read(catalog)
        xt = cat[cat['id']==gid]['x'].data[0]
        yt = cat[cat['id']==gid]['y'].data[0]
        x.append(xt)
        y.append(yt)
        
    return x, y

In [26]:
#Get boundaries for .feedme file (with 100pixel border at each edge)
def get_boundaries(gids, field):
    
    field = [field]*len(gids)
    
    x, y = get_positions(gids, field)

    d = {'gid':gids, 'field':field, 'xpos':x, 'ypos':y}
    df = pd.DataFrame(data=d)

    xmin = int(df['xpos'].min())-100
    ymin = int(df['ypos'].min())-100
    xmax = int(df['xpos'].max())+100
    ymax = int(df['ypos'].max())+100
    
    return xmin, xmax, ymin, ymax

In [123]:
#Get objects nearby objects being fit to improve the fit
def get_nearby_objects(gid, field):
    
    if field == 'S':
        #Find catalog
        catalog = '/Users/rosaliaobrien/research/website/catalogs/goodss_3dhst.v4.4.cat'
        cat = ascii.read(catalog)
        
    if field == 'N':
        #Find catalog
        catalog = '/Users/rosaliaobrien/research/website/catalogs/goodsn_3dhst.v4.4.cat'
        cat = ascii.read(catalog)
        
    x = cat[cat['id']==gid]['x']
    y = cat[cat['id']==gid]['y']
    
    objects = cat[(x-70<cat['x']) & (cat['x']<x+70) & (y-70<cat['y']) & (cat['y']<y+70)]
    o = objects['id'].data
    
    for i in o:
        if len(str(i)) > 5:
            o = np.delete(o, np.where(o==i)[0][0])
        
    return o

In [28]:
#Write intro of .feedme file
def write_intro(gids, field, flter, outputfile):
    
    outputfilefits = outputfile+'.fits'
    
    xmin, xmax, ymin, ymax = get_boundaries(gids, field)
    
    if field =='S':
        fitsfile = 'goodss_3dhst.v4.0.F'+flter+'W_orig_sci.fits'
        inputpsf = 'goodss_3dhst.v4.0.F'+flter+'W_psf.fits'
        psffine = 'goodss_3dhst.v4.0.F'+flter+'W_orig_wht.fits'
        
    if field =='N':
        fitsfile = 'goodsn_3dhst.v4.0.F'+flter+'W_orig_sci.fits'
        inputpsf = 'goodsn_3dhst.v4.0.F'+flter+'W_psf.fits'
        psffine = 'goodsn_3dhst.v4.0.F'+flter+'W_orig_wht.fits'
        
    
    text = '''================================================================================\n\
# IMAGE and GALFIT CONTROL PARAMETERS\n\
A) {0}           # Input data image (FITS file)\n\
B) {1}     					  # Output data image block\n\
C) none              					  # Sigma image name (made from data if blank or "none") \n\
D) {2}       # Input PSF image and (optional) diffusion kernel\n
E) none                                   # PSF fine sampling factor relative to data\n 
F) none              					  # Bad pixel mask (FITS image or ASCII coord list)\n\
G) none               					  # File with parameter constraints (ASCII file) \n\
H) {4} {5} {6} {7}       # Image region to fit (xmin xmax ymin ymax)\n\
I) 100    100        					  # Size of the convolution box (x y)\n\
J) 25.7551            					  # Magnitude photometric zeropoint \n\
K) 0.038  0.038      					  # Plate scale (dx dy)   [arcsec per pixel]\n\
O) regular           					  # Display type (regular, curses, both)\n\
P) 0                 					  # Options: 0=normal run; 1,2=make model/imgblock & quit
\n'''.format(fitsfile, outputfilefits, inputpsf, psffine, xmin, xmax, ymin, ymax)
    
    return text

In [22]:
#Print out object info for .feedme file
def write_object(gid, field, coord, outputfile):

    if field == 'S':
        #Find catalog
        catalog = '/Users/rosaliaobrien/research/website/catalogs/goodss_3dhst.v4.4.cat'
        cat = ascii.read(catalog)
        
    if field == 'N':
        #Find catalog
        catalog = '/Users/rosaliaobrien/research/website/catalogs/goodsn_3dhst.v4.4.cat'
        cat = ascii.read(catalog)
    
    #Estimate effective radius from catalog
    radius = cat[cat['id']==gid]['flux_radius'].data[0]/2
    
    #Estimate axis raio from catalog and round to 2 decimal places
    axis_ratio = '{:.2f}'.format(cat[cat['id']==gid]['b_image'].data[0]/cat[cat['id']==gid]['a_image'].data[0])
    
    #Print text for .feedme file
    text='#ID: {0} \n\
0) sersic            		    # Object type \n\
 1) {1}  1 1                    # position x, y        [pixel]\n\
 3) 20     1    		        # total magnitude    \n\
 4) {2}  1    		        #     R_e              [Pixels]\n\
 5) 2.00   1                    # Sersic exponent (deVauc=4, expdisk=1)  \n\
 9) {3}   1                    # axis ratio (b/a)   \n\
10) 45     1                    # position angle (PA)  [Degrees: Up=0, Left=90] \n\
\n'.format(gid, coord, radius, axis_ratio)

    with open(outputfile+'.feedme', "a") as myfile:
        myfile.write(text)

In [23]:
#Write .feedme file for object and it's nearby objects
def write_feedme(gid, field, flter, outputfile):
    
    nearby_objects = get_nearby_objects(gid, field)
    objects = nearby_objects
    fields = [field]*len(objects)
    
    print('Writing intro...')
    text = write_intro(objects, field, flter, outputfile)
    save = outputfile+'.feedme'
    np.savetxt(save, np.array(text).reshape(1, ), fmt='%s')
    
#     coord = get_positions(fields, objects)
#     write_object(gid, field, coord, outputfile)

#     print('Creating object number', gids.index(i)+1)

#     objects.append(i)

    for i in objects:
        coord = get_position(field, i)
        write_object(i, field, coord, outputfile)

        print('Writing object '+str(i)+'...')
#         objects.append(ii)
            
#     print('Number of objects:', len(objects))

In [9]:
# catalogs = '/Users/rosaliaobrien/research/website/catalogs/goodss_3dhst.v4.4.cat'
# catalogn = '/Users/rosaliaobrien/research/website/catalogs/goodsn_3dhst.v4.4.cat'
# cats = ascii.read(catalogs)
# catn = ascii.read(catalogn)

In [136]:
#Objects from Vince's paper

gids = [23758, 37955, 16758, 43615, 42221, 39241, 45972, 44620, 
        39631, 39170, 34694, 23435, 47677, 39805, 38785, 32566, 
        40476, 21156, 17070, 35774, 40597, 37686, 46066, 40862, 
        39804, 21427, 40623, 41520, 40223, 39012, 44042]

fields = ['N', 'N', 'N', 'S', 'S', 'S', 'S', 'S',
          'S', 'S', 'N', 'N', 'S', 'S', 'S', 'N', 
          'S', 'N', 'N', 'S', 'S', 'N', 'S', 'S', 
          'S', 'N', 'S', 'S', 'S', 'S', 'S']

In [137]:
#Separate GOODS N and GOODS S

gidsn =[]
gidss =[]

for field, gid in zip(fields, gids):
    if field == 'N':
        gidsn.append(gid)
        
    else:
        gidss.append(gid)

In [149]:
def run_galfit(gids, field, flter, write_files=False):
    #Field must be 'S' or 'N'
    if not (field == 'S' or field == 'N'):
        raise Exception('Field must be S or N.')
        
    if not type(flter) == str:
        raise Exception('Filter must be string.')
    
    os.chdir('/Users/rosaliaobrien/research/science_project/gal_morphology/North/'+flter+'/')
    galfit = '/Users/rosaliaobrien/Bin/galfit'

    #Run for all gids in specfic field
    for gid in gids:
        print('\n***RUNNING OBJECT '+str(gid)+'***')

        if write_files == True:
            #write each feedme file
            write_feedme(gid, field, flter, str(gid))
            
        else:
            os.rename(field+'/'+str(gid)+'.feedme', str(gid)+'.feedme')

        print('Running GALFIT...')
        #run galfit for each feedme file!
        subprocess.call([galfit, str(gid)+'.feedme'])

        if not os.path.exists(field):
            os.mkdir(field)
            
        if not os.path.exists(field+'/fits'):
            os.mkdir(field+'/fits')
            
        if not os.path.exists(field+'/'+str(gid)):
            os.mkdir(field+'/'+str(gid))
        
        feedme = str(gid)+'.feedme'
        fits = str(gid)+'.fits'
        galfitrun = 'galfit.01'
        
        if os.path.exists(fits):
            os.rename(str(gid)+'.fits', field+'/fits/'+str(gid)+'.fits')
            
        if os.path.exists(feedme):
            os.rename(str(gid)+'.feedme', field+'/'+str(gid)+'/'+str(gid)+'.feedme')
            
        if os.path.exists(galfitrun):
            galfitrun = os.rename('galfit.01', field+'/'+str(gid)+'/galfit.01')

In [150]:
run_galfit(gidsn, 'N', '160', write_files=True)


***RUNNING OBJECT 23758***
Running GALFIT...

***RUNNING OBJECT 37955***
Running GALFIT...

***RUNNING OBJECT 16758***
Running GALFIT...

***RUNNING OBJECT 34694***
Running GALFIT...

***RUNNING OBJECT 23435***
Running GALFIT...

***RUNNING OBJECT 32566***
Running GALFIT...

***RUNNING OBJECT 21156***
Running GALFIT...

***RUNNING OBJECT 17070***
Running GALFIT...

***RUNNING OBJECT 37686***
Running GALFIT...

***RUNNING OBJECT 21427***
Running GALFIT...
